In [ ]:
from dash import Dash, dash_table, dcc, callback, Output, Input
import pandas as pd
import plotly.express as px
import dash_mantine_components as dmc
from requests import get
from dash.dependencies import MATCH, ALL, State
from dash.exceptions import PreventUpdate
from dash import callback_context
from dash import html

In [ ]:
df = pd.read_csv("C:/Users/kaila/OneDrive/Desktop/COMP3610PROJECT/notebooks/csv's/CPS_Summary.csv")

df.head()

In [ ]:
app = Dash()

app.layout = dmc.MantineProvider(
    dmc.Container(
        [
            dmc.Title("Brand per Category Popularity Score", 
                      style={"size":"h3","color": "blue", "textAlign": "center"}), 

            dmc.Select(
                id="category-dropdown",
                data=[{"label": cat, "value": cat} for cat in df["category"].unique()],
                placeholder="Select a category",
                searchable=True,
                clearable=True,
            ),

            dmc.Space(h=20),

            html.Div(id="time-series-container")
        ], fluid=True
    )
)
@callback(
    Output(component_id="time-series-container", component_property="children"),
    Input(component_id="category-dropdown", component_property="value")
)
def update_time_series(selected_category):
    if not selected_category:
        return "Please select a category."
    
    # Filter by selected category
    filtered_df = df[df["category"] == selected_category]

    # Example: create a "date" from year and month for x-axis
    filtered_df = filtered_df.copy()
    filtered_df["date"] = pd.to_datetime(filtered_df[["year", "month"]].assign(day=1))

    # Plot using the new 'date' column and some value (e.g., 'BPCPS')
    fig = px.line(filtered_df, x="date", y="CPS", 
                  title=f"Time Series for Category {selected_category}")
    
    return dcc.Graph(figure=fig)

if __name__ == "__main__":
    app.run(debug=True)